In [0]:
%python
# Define the path to the CSV file
csv_file_path = "/Volumes/workspace/globalretail/customer"

# Read the CSV file into a DataFrame
customer_df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(csv_file_path)

# Display the DataFrame
display(customer_df)

In [0]:
from pyspark.sql.functions import current_timestamp
df_new = customer_df.withColumn("ingestion_timestamp", current_timestamp())
display(df_new)

In [0]:
%python
# Set the schema to globalretail
spark.sql("USE globalretail")

# Define the DataFrame (example definition, replace with actual data loading logic)
customer_df = spark.read.format("csv").option("header", "true").load("/Volumes/workspace/globalretail/customer")

# Write the DataFrame to a Delta table
customer_df.write.format("delta").mode("append").saveAsTable("bronze_customer")

In [0]:
spark.sql("select * from bronze_customer limit 100").show()